In [1]:
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

/tmp/ipykernel_2823735/254707315.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
PER_PACKET_COLS = ['_time', 'packet_interarrival_time', 'packet_jitter_raw', 'packet_jitter_weighted', 'packet_owd']
AGGREGATE_COLS  = ['_time', 'availability_2ms', 'availability_4ms', 'availability_8ms', 'availability_16ms', 'availability_32ms', 'availability_64ms', 'availability_128ms', 'packet_loss', 'throughput']

In [3]:
FILE_PATH = "/home/shared/validation_files/csvs/results"
df1, df2, df3 = pd.read_csv(f"{FILE_PATH}/analysis_validation_one.csv", skiprows=3),\
                pd.read_csv(f"{FILE_PATH}/analysis_validation_two.csv", skiprows=3),\
                pd.read_csv(f"{FILE_PATH}/analysis_validation_three.csv", skiprows=3)

df_aggregate1, df_aggregate2, df_aggregate3 = pd.read_csv(f"{FILE_PATH}/analysis_validation_one_aggregate.csv", skiprows=3),\
                pd.read_csv(f"{FILE_PATH}/analysis_validation_two_aggregate.csv", skiprows=3),\
                pd.read_csv(f"{FILE_PATH}/analysis_validation_three_aggregate.csv", skiprows=3)

In [4]:
df1 = df1[PER_PACKET_COLS]
df2 = df2[PER_PACKET_COLS]
df3 = df3[PER_PACKET_COLS]

df_aggregate1 = df_aggregate1[AGGREGATE_COLS]
df_aggregate2 = df_aggregate2[AGGREGATE_COLS]
df_aggregate3 = df_aggregate3[AGGREGATE_COLS]

In [5]:
df1.head()

,_time,packet_interarrival_time,packet_jitter_raw,packet_jitter_weighted,packet_owd
0,1970-01-01T00:00:00.001Z,0.001,0.002187,0.000137,0.009830
1,1970-01-01T00:00:00.002Z,0.001,0.000408,0.000154,0.010238
2,1970-01-01T00:00:00.003Z,0.001,0.000055,0.000147,0.010293
3,1970-01-01T00:00:00.004Z,0.001,0.005730,0.000496,0.016023
4,1970-01-01T00:00:00.005Z,0.001,-0.013857,0.001331,0.002167


In [6]:
df_aggregate1.head()

,_time,availability_2ms,availability_4ms,availability_8ms,availability_16ms,availability_32ms,availability_64ms,availability_128ms,packet_loss,throughput
0,1970-01-01T00:00:00Z,0.013,0.050,0.196,0.786,0.988,0.988,0.988,0.008,460288
1,1970-01-01T00:00:01Z,0.019,0.054,0.196,0.759,0.986,0.986,0.986,0.005,461680
2,1970-01-01T00:00:02Z,0.011,0.037,0.183,0.764,0.978,0.978,0.978,0.011,458896
3,1970-01-01T00:00:03Z,0.008,0.037,0.163,0.744,0.979,0.979,0.979,0.013,457968
4,1970-01-01T00:00:04Z,0.008,0.032,0.172,0.758,0.982,0.982,0.982,0.010,459360


In [7]:
def parse_timestamp(timestamp_str, with_fraction=True):
    if with_fraction:
        try:
            seconds_part = timestamp_str[:-1].split('.')[0]
            fraction_part = timestamp_str[:-1].split('.')[1]
        except:
            seconds_part = timestamp_str[:-1]
            fraction_part = 0
    else:
        seconds_part = timestamp_str[:-1]

    timestamp = datetime.strptime(seconds_part, '%Y-%m-%dT%H:%M:%S')

    return timestamp.timestamp() + (float(f"0.{fraction_part}") if with_fraction else 0)

In [8]:
df1['_time'] = df1['_time'].apply(parse_timestamp)
df2['_time'] = df2['_time'].apply(parse_timestamp)
df3['_time'] = df3['_time'].apply(parse_timestamp)

df_aggregate1['_time'] = df_aggregate1['_time'].apply(lambda x: parse_timestamp(x, False))
df_aggregate2['_time'] = df_aggregate2['_time'].apply(lambda x: parse_timestamp(x, False))
df_aggregate3['_time'] = df_aggregate3['_time'].apply(lambda x: parse_timestamp(x, False))

In [9]:
def analyze_iat(df):
    iat = df['packet_interarrival_time']
    mean, std = iat.mean(), iat.std()

    return mean, std

def analyze_owd(df):
    owd = df['packet_owd']
    mean, std = owd.mean(), owd.std()

    return mean, std

def analyze_packet_loss(df):
    packet_loss = df['packet_loss']
    return packet_loss.mean()

def analyze_throughput(df):
    throughput = df['throughput']
    return throughput.mean()

In [10]:
print("DF1 IAT")
print(f'Expected mean: 0.001, Calculated mean: {analyze_iat(df1)[0]}')
print(f'Expected std: 0.0, Calculated std: {analyze_iat(df1)[1]}')
print("\n")

print("DF2 IAT")
print(f'Expected mean: 0.001, Calculated mean: {analyze_iat(df2)[0]}')
print(f'Expected std: 0.0, Calculated std: {analyze_iat(df2)[1]}')
print("\n")

print("DF3 IAT")
print(f'Expected mean: 0.001, Calculated mean: {analyze_iat(df3)[0]}')
print(f'Expected std: 0.0, Calculated std: {analyze_iat(df3)[1]}')
print("\n")

DF1 IAT
Expected mean: 0.001, Calculated mean: 0.000999999999999938
Expected std: 0.0, Calculated std: 3.546330235984574e-14


DF2 IAT
Expected mean: 0.001, Calculated mean: 0.000999999999999938
Expected std: 0.0, Calculated std: 3.546330235984574e-14


DF3 IAT
Expected mean: 0.001, Calculated mean: 0.000999999999999938
Expected std: 0.0, Calculated std: 3.546330235984574e-14




In [11]:
print("DF1 OWD")
print(f'Expected mean: 0.012, Calculated mean: {analyze_owd(df1)[0]}')
print(f'Expected std: 0.005, Calculated std: {analyze_owd(df1)[1]}')
print("\n")

print("DF2 OWD")
print(f'Expected mean: 0.032, Calculated mean: {analyze_owd(df2)[0]}')
print(f'Expected std: 0.025, Calculated std: {analyze_owd(df2)[1]}')
print("\n")

print("DF3 OWD")
print(f'Expected mean: 0.02, Calculated mean: {analyze_owd(df3)[0]}')
print(f'Expected std: 0.05, Calculated std: {analyze_owd(df3)[1]}')
print("\n")

DF1 OWD
Expected mean: 0.012, Calculated mean: 0.012118981144254354
Expected std: 0.005, Calculated std: 0.004858362344498837


DF2 OWD
Expected mean: 0.032, Calculated mean: 0.03199234182478367
Expected std: 0.025, Calculated std: 0.007500764103308245


DF3 OWD
Expected mean: 0.02, Calculated mean: 0.025753960715401886
Expected std: 0.05, Calculated std: 0.01586684312171544




In [12]:
print("DF1 Packet loss")
print(f'Expected rate: 0.01, Calculated rate {analyze_packet_loss(df_aggregate1)}')
print("\n")

print("DF2 Packet loss")
print(f'Expected rate: 0.05, Calculated rate {analyze_packet_loss(df_aggregate2)}')
print("\n")

print("DF1 Packet loss")
print(f'Expected rate: 0.2, Calculated rate {analyze_packet_loss(df_aggregate3)}')
print("\n")

DF1 Packet loss
Expected rate: 0.01, Calculated rate 0.009818000032333615


DF2 Packet loss
Expected rate: 0.05, Calculated rate 0.04999300007894631


DF1 Packet loss
Expected rate: 0.2, Calculated rate 0.19929699984192847




In [13]:
print("DF1 Throughput")
print(f'Expected rate: 0.01, Calculated rate {analyze_throughput(df_aggregate1)}')
print("\n")

print("DF2 Throughput")
print(f'Expected rate: 0.05, Calculated rate {analyze_throughput(df_aggregate2)}')
print("\n")

print("DF1 Throughput")
print(f'Expected rate: 0.2, Calculated rate {analyze_throughput(df_aggregate3)}')
print("\n")

DF1 Throughput
Expected rate: 0.01, Calculated rate 459444.448


DF2 Throughput
Expected rate: 0.05, Calculated rate 440803.248


DF1 Throughput
Expected rate: 0.2, Calculated rate 371526.192


